In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import csv
import math
import jieba

In [ ]:
text1 = "It's good, Jinbao says I would like to come again."
text2 = "It's good, Meer says I would like to come again it good."
text3 = "It might be come an excellent one. But I won't come again."
text4 = "It's good, Ginny says I would like to come again."
text5 = "It's good, Penny says I would like to come again."
text6 = "It's good, BadB says I would like to come again."
corpus = tuple([text1, text2, text3, text4, text5, text6])


counter = CountVectorizer(ngram_range=(1,2))
langMod = counter.fit_transform(corpus)

# langMod.toarray()
print(counter.get_feature_names())
query = "It's good"#, Jinbao says I would like to come again."
print('> counter.transform([query]).toarray()')
print(counter.transform([query]).toarray())
print('> langMod.toarray()')
print(langMod.toarray())
print('> Test P( again | come )')
loc0 = counter.vocabulary_.get('come')
loc1 = counter.vocabulary_.get('come again')

print(sum(langMod.toarray().transpose()[loc1]) / sum(langMod.toarray().transpose()[loc0]))

In [ ]:
counter = CountVectorizer(ngram_range=(1,2))
langMod = counter.fit_transform(corpus)
langModTrans = langMod.toarray().transpose()

def condProb(bigram, langModTrans):
    cond = bigram.split(' ')[0]
    loc0 = counter.vocabulary_.get(cond)
    loc1 = counter.vocabulary_.get(bigram)
    prob = sum(langModTrans[loc1]) / sum(langModTrans[loc0])
    return prob
# condProb('come again', langModTrans)

def parseSeq(query):
    counterTemp = CountVectorizer(ngram_range=(2,2))
    langModTemp = counterTemp.fit_transform([query])
    bigrams = counterTemp.get_feature_names()
    return bigrams

# text1 = "It's good, Jinbao says I would like to come again."
# parseSeq(text2)

def sequenceProb(query):
    bigrams = parseSeq(query)
#     for bigram in bigrams:
#         print( -math.log(condProb(bigram, langModTrans)))
    logProb = sum(map(lambda x: math.log(condProb(x, langModTrans)), bigrams))
    print(math.exp(logProb))
    #     print( sum([math.log(condProb(x, langModTrans)) for x in bigrams]) )

sequenceProb(text1)
sequenceProb(text2)
sequenceProb(text3)
sequenceProb(text4)
sequenceProb(text5)

In [ ]:
print(math.log(0.09))
print(math.log(0.9) + math.log(0.1))
print(math.log(0.01))
print(math.log(0.1) + math.log(0.1))

print(sum(map(math.log, [1,3,5])))
a = math.log(0.1) + math.log(0.3) + math.log(0.5)
print(a)
math.exp(a)

In [14]:
data0109 = pd.read_csv('data0109.csv')
records = data0109['new_GeneralInformation']

In [24]:
records = records.dropna()
records[7]

'1. 該司12/19報名2018國際遊艇展2. 選位時，希望與高雄市觀光局攤位等相鄰，互相吸引人潮。也協助本處強化會展旅遊陣容3. 該司12/22電子郵件發2018國際遊艇展承辦人張心怡，表達希望進一步策略合作4. 該司提議交換資源應用，例如:    A.提供VIP招待券(價值$380)或    B.貴賓團參觀附導覽(每場40人/價值$15,200元)5. 相對希望貿協    A.攤位費用交換或    B.廣告版面交換或    C.遊艇展參觀票券'

In [27]:
jieba.load_userdict("/usr/local/lib/python3.5/dist-packages/jieba/user.dict")
corpus = []
for record in records:
    seg_list = jieba.cut(record, cut_all=False)
    corpus.append(' '.join(seg_list))
corpus


['一 、 臺灣 行銷 科學學會 （ 以下 簡稱 該 學會 ） 任 秘書長 立中 表示 該學會業 已 爭取 「 2019 年亞洲 行銷聯盟 大會 」 來 臺 辦理 ， 並擬 於 同時間 辦理 首屆 「 臺灣 行銷 及 品牌 博覽會 （ Taiwan   Marketing   and   Branding   Expo ） 」 。 二 、 「 2019 年亞洲 行銷聯盟 大會 」 暫訂 於 2019 年 3 月 辦理 ， 除亞洲 行銷聯盟 會 理事會 、 會員 大會外 ， 將 結合 大師 講座 （ World   Marketing   Summit ） 、 全球 華人 行銷聯盟 （ Global   Chinese   Marketing   Federation ） 研討會 及 產業 分組 研討會 辦理 。 三 、 有鑑 於 國內 並無行 銷及 品牌 相關 展覽 ， 為 提升 我國 中小 企業 品牌 行銷 能力 ， 任 秘書長 期可集 結產 、 官 、 學界 力量 ， 辦理 首屆 B2B 「 臺灣 行銷 及 品牌 博覽會 （ Taiwan   Marketing   and   Branding   Expo ） 」 。 初步 規劃 徵集 400 家國 內外 品牌 行銷 相關業者 ， 如公關 公司 、 形象 出版 公司 、 CRM 公司 、 物流 通路 等 共同 展出 。 四 、 為 提升 活動 整體 效益 ， 任 秘書長 表示 目前 規劃 以會 加展 方式 辦理 ， 會議 場地 以 臺北國際會議中心 為主 ， 展覽 部份 則 希望 使用 鄰近 之 世貿一館 。 五 、 本 中心 已向 該會 介紹 MEET   TAIWAN 服務 項目 ， 將後續 協助 洽詢 會議 及 展覽 場地 檔期 ， 並 彙 整 MEET   TAIWAN 與 其他 政府 機關 補捐助 資訊 。',
 '1 .   該司 產品 住 要 供 海事 使用 。 民用 產品 很少 2 .   已 決定 參加 2018 「 高雄國際 海事 船舶 暨國防 工業展 」 ， 不 考慮 遊艇展',
 '整體 服務 滿意 , 內容 實用',
 '( 一 ) 該 公司 與 日本 近畿 旅行社 策略 合作 ， 專營 日本 市場 ， 業務以 觀光 為 主 ， 亦 承接 日本 企業 獎勵 旅遊 、 教育 旅行 、 

In [ ]:
import jieba.analyse

jieba.analyse.extract_tags(records[40], topK=3, withWeight=False, allowPOS=())

In [ ]:
counter = CountVectorizer(ngram_range=(1,2))
langMod = counter.fit_transform(corpus)
langModTrans = langMod.toarray().transpose()

def condProb(bigram, langModTrans):
    cond = bigram.split(' ')[0]
    loc0 = counter.vocabulary_.get(cond)
    loc1 = counter.vocabulary_.get(bigram)
    print('{} / {}'.format(sum(langModTrans[loc1]), sum(langModTrans[loc0])))
    prob = sum(langModTrans[loc1]) / sum(langModTrans[loc0])
    return prob
# condProb('come again', langModTrans)

def parseSeq(query):
    counterTemp = CountVectorizer(ngram_range=(2,2))
    langModTemp = counterTemp.fit_transform([query])
    bigrams = counterTemp.get_feature_names()
    return bigrams

# text1 = "It's good, Jinbao says I would like to come again."
# parseSeq(text2)

def sequenceProb(query):
    bigrams = parseSeq(query)
#     for bigram in bigrams:
#         print( -math.log(condProb(bigram, langModTrans)))
    logProb = sum(map(lambda x: math.log(condProb(x, langModTrans)), bigrams))
    print(math.exp(logProb))
    #     print( sum([math.log(condProb(x, langModTrans)) for x in bigrams]) )

sequenceProb(corpus[0])
sequenceProb(corpus[1])
sequenceProb(corpus[2])
sequenceProb(corpus[3])
sequenceProb(corpus[4])

In [ ]:
# condProb('前來 本處', langModTrans)
condProb('巴西 日本', langModTrans)
condProb('海外 機率', langModTrans)


In [ ]:
for i in range(1,10):
    print('')
    for j in range(1,10):
        print('%2d' % (i * j), end=' ')

for i in range(100):
    if int(i % 10) == 0: print('')
    if int(i / 10) * (i % 10) != 0:
        print('%2d' % (int(i / 10) * (i % 10)), end=' ')

In [4]:
# 1 , 1 , 2 , 3 , 5 , 8 , 13 , 21 , 34 , 55 , 89 , 144
def main(n):
    def feb(n):
        if n == 0:
            return 1
        if n == 1:
            return 1
        if febList[n]:
            return febList[n]
        ans = feb(n - 1) + feb(n - 2)
        febList[n] = ans
        return ans
    
    febList = [0] * (n+1)
    answer = feb(n)
    print(febList)
    return answer
main(144)

[0, 0, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

898923707008479989274290850145

In [ ]:
febList = [0] * 10
febList[3] = 1
if not febList[2]:
    print('yes')